### Run all this before playing


In [ ]:
%pip install qiskit
%pip install qiskit-aer
%pip install pylatexenc
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
import numpy as np
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from math import factorial
import random
import math
import time
from qiskit import transpile

In [ ]:
def game(pair, players):




    while True:
        #####check for zeros

        count = 0
        for i in pair:
            if pair[i][1] == 0:
                count += 1
            else:
                ans = i
        if count == 3:
            return f'{ans} IS A WINNER'

        ###############process #################


        ###############setting up the circuit##########
        labels = []
        for i in range(players):
            for j in range(pair[f'Player {i+1}'][1]):
                labels.append(f'Player_{i+1} -> Dice_{j+1}')



        dicts = {i:QuantumRegister(2, i) for i in labels}

        qc_labels = list(dicts.values())




        qout = QuantumRegister(1, 'output')
        c = ClassicalRegister(2*dice_count*players)

        qc = QuantumCircuit(qout, *qc_labels, c)

        stv = Statevector.from_label('0'*2*dice_count*players)

        for i in qc_labels:
            for j in range(len(i)):
                qc.h(i[j])

        record = [{f'Player {(2*i) + 1}': i_to_b(pair[f'Player {(2*i) + 1}'][0]), f'Player {(2*i) + 2}': i_to_b(pair[f'Player {(2*i) + 2}'][0])} for i in range(players // 2)]

        ######################collusion##################
        for i in range(players//2):
            if pair[f'Player {(2*i) + 1}'][1] != 0 and pair[f'Player {(2*i) + 2}'][1] != 0:
                #######control############
                lst = [j for j in labels if f'Player_{(2*i) + 1}' in j]
                choice = random.choice(lst)
                player_1_1 = dicts[choice][0]
                player_1_2 = dicts[choice][1]
                ########################
                ###########target###########
                lst = [j for j in labels if f'Player_{(2*i) + 2}' in j]
                choice = random.choice(lst)
                player_2_1 = dicts[choice][0]
                player_2_2 = dicts[choice][1]


                ###################
                #step one, apply nots if necessary on control
                ent = list(record[i][f'Player {(2*i) + 1}'])
                entd = list(record[i][f'Player {(2*i) + 2}'])
                #dice 0 --> [1]
                if ent[1] == '0':
                    qc.x(player_1_1)
                if ent[0] == '0':
                    qc.x(player_1_2)


                #step two apply the controlled hadards
                qc.cu(math.pi/2, 0, math.pi, 0, player_1_1, player_2_1)
                qc.cu(math.pi/2, 0, math.pi, 0, player_1_2, player_2_2)

                #step three apply the nots
                if ent[1] == '0':
                    qc.x(player_1_1)
                if ent[0] == '0':
                    qc.x(player_1_2)


                if entd[1] == '1':
                    qc.x(player_2_1)
                if entd[0] == '1':
                    qc.x(player_2_2)
        qc.barrier()
        for i in qc_labels:
            for j in range(len(i)):
                qc.measure(i[j], c[count])
                count += 1


        qc.draw(output = 'mpl')
        backend = Aer.get_backend('aer_simulator')



        counts= backend.run(qc, shots=1).result().get_counts(qc)












        #val = format(max(lst), 'b').zfill(sum([pair[j][1] for j in pair]))
        bit_string = [i for i in counts.keys()][0]

        bite_string = bit_string[::-1]

        bitt_string = [bite_string[i:i+2][::-1] for i in range(0, len(bite_string), 2)]

        vals = {}
        count = 0
        for i in range(len(pair)):
            vals[f'Player {i+1}'] = 'confirm'
            for j in range(1, pair[f'Player {i+1}'][1]+1):
                vals[(f'Player {i+1}', f'Dice {j}')] = int(bitt_string[count],2)+1
                count += 1



        for i in vals:
            if vals[i] == 'confirm':
                input(f'Press Enter to reveal {i}\'s dice')
                continue
            print(f'{i} ---> {vals[i]}')

        lost = 0
        while lost != 1 and lost != 2 and lost != 3 and lost != 4:
            lost = int(input('please enter which player lost (type 1/2/3/...)  \n'))

        pair[f'Player {lost}'][1] -= 1






    #######someone loses a dice


### Setup

Run the code block below to declare the number of dice per player:

In [ ]:
initial_players = 4
dice_count = int(input('how many 4-sided dice per player\n (no more than 3)\n'))
def i_to_b(x):
    a = format(x-1, 'b')
    if len(a) < 2:
        for _ in range(2-len(a)):
            a = '0'+a
    return a

Run the code block below to collude:

In [ ]:
pair = {}


for i in range(initial_players//2):
    entangler = ''
    entangled = ''
    while entangler != '1' and entangler != '2' and entangler != '3' and entangler != '4' and entangler != 'N/A':
        entangler = input(f'what value do you wish to act as a control value (Player {2*(i+1) - 1} --> {2*(i+1)}) ?\n please type values 1 to 4 or N/A\n')
    if entangler != 'N/A':
        while entangled != '1' and entangled != '2' and entangled != '3' and entangled != '4' and entangled != 'N/A':
            entangled = input(f'what value do you wish to act as a target value (Player {2*(i+1)}), controlled by the control value\n')
    pair[f'Player {2*(i+1) - 1}'] = [int(entangler)]
    pair[f'Player {2*(i+1)}'] = [int(entangled)]

for i in pair:
    pair[i].append(dice_count)

### PLAY!

The program will give each player's dice rolls separately which are to be delivered by a neutral third party to each player.

After hearing their rolls, the players must bid. You bid by declaring a specific amount of a specific dice roll, like four 2s. You can raise another person's bid by increasing the number of dice in the previous bid, like raising three 2s to four 2s, or by increasing the specific dice roll. Note that when increasing the specific dice roll, you can decrease the number of dice claimed. For example, you can raise five 3s to three 4s. You can challenge a bid and once a challenge is made, no more bids are allowed. If there are less than the specific amount(of the specific dice value) claimed in the bid, the last bidder loses and if there are at least that many, the challenger loses. For example if the last bid was four 5s and there actually was three 5s, the last bidder loses but if there were four or more 5s, the challenger loses. Report to the program who lost the bidding session

If you lose, you will lose a die. The game will go until there is only 1 player with dice remaining who is the winner.

In [ ]:
game(pair, initial_players)